In [1]:
pip install -r ../../requirements.txt

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [3]:
%run -i ../../zkstats/core.py

In [16]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

In [23]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

data = np.array(json.loads(open(data_path, "r").read())['col_name'])

create_dummy(data_path, dummy_data_path)
dummy_data = np.array(json.loads(open(dummy_data_path, "r").read())['col_name'])

dummy_theory_output = torch.tensor(np.median(dummy_data[dummy_data<50]))
dummy_lower_to_median = torch.tensor(np.sort(dummy_data[dummy_data<50])[int(len(dummy_data[dummy_data<50])/2)-1])
dummy_upper_to_median = torch.tensor(np.sort(dummy_data[dummy_data<50])[int(len(dummy_data[dummy_data<50])/2)])

theory_output = torch.tensor(np.median(data[data<50]))
lower_to_median = torch.tensor(np.sort(data[data<50])[int(len(data[data<50])/2)-1])
upper_to_median = torch.tensor(np.sort(data[data<50])[int(len(data[data<50])/2)])


In [ ]:
scales = [2]
selected_columns = ['col_name']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [39]:
# Verifier/ data consumer side: send desired calculation
class verifier_model(nn.Module):
    def __init__(self):
        super(verifier_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = dummy_theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = dummy_lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = dummy_upper_to_median, requires_grad = False)
    def forward(self,X):
        # where part
        # need to initiate num_lowest (number of values that get filtered out) before num_fil_X due to constraint behavior
        filter = X<50
        num_lowest = X.size()[1]-torch.sum(filter.double())
        num_fil_X = X.size()[1]-num_lowest
        min_X = torch.min(X)
        fil_X = torch.where(filter, X, min_X-1)

        count_less = torch.sum((fil_X < 0.99*self.w).double()) - num_lowest
        count_equal = torch.sum((torch.abs(fil_X-self.w)<=torch.abs(0.01*self.w)).double())
        half_len = torch.floor(torch.div(num_fil_X, 2))

        # not support modulo yet
        less_cons = count_less<half_len+2*(num_fil_X/2 - torch.floor(num_fil_X/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0 --> imply even length for sure
        lower_exist = torch.sum((torch.abs(fil_X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((fil_X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(fil_X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((fil_X<0.99*self.upper).double()) - num_lowest==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))

        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)


verifier_define_calculation(dummy_data_path, selected_columns,sel_dummy_data_path,verifier_model, verifier_model_path)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/symbolic_opset9.py:2174: FutureWarning: 'torch.onnx.symbolic_opset9._cast_Bool' is deprecated in version 2.0 and will be removed in the future. Please Avoid using this function and create a Cast node instead.
  return fn(g, to_cast_func(g, input, False), to_cast_func(g, other, False))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/onnx/utils.py:1686: UserWarning: The exported ONNX model failed ONNX shape inference. The model will not be executable by the ONNX Runtime. If this is unintended and you believe there is a bug, please report an issue at https://github.com/pytorch/pytorch/issues. Error reported by strict ONNX shape inference: [ShapeInferenceError] (op_type:Where, node name: /Where): Y has inconsistent type tensor(float) (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/jit/serialization/export.cpp:1421.)
  _C._check_onnx

In [40]:
# prover calculates settings, send to verifier
print("Theory_output: ", theory_output)
class prover_model(nn.Module):
    def __init__(self):
        super(prover_model, self).__init__()
        # w represents mean in this case
        self.w = nn.Parameter(data = theory_output, requires_grad = False)
        self.lower = nn.Parameter(data = lower_to_median, requires_grad = False)
        self.upper = nn.Parameter(data = upper_to_median, requires_grad = False)
    def forward(self,X):
        # where part
        # need to initiate num_lowest (number of values that get filtered out) before num_fil_X due to constraint behavior
        filter = X<50
        num_lowest = X.size()[1]-torch.sum(filter.double())
        num_fil_X = X.size()[1]-num_lowest
        min_X = torch.min(X)
        fil_X = torch.where(filter, X, min_X-1)

        count_less = torch.sum((fil_X < 0.99*self.w).double()) - num_lowest
        count_equal = torch.sum((torch.abs(fil_X-self.w)<=torch.abs(0.01*self.w)).double())
        half_len = torch.floor(torch.div(num_fil_X, 2))

        # not support modulo yet
        less_cons = count_less<half_len+2*(num_fil_X/2 - torch.floor(num_fil_X/2))
        more_cons = count_less+count_equal>half_len

        # For count_equal == 0 --> imply even length for sure
        lower_exist = torch.sum((torch.abs(fil_X-self.lower)<=torch.abs(0.01*self.lower)).double())>0
        lower_cons = torch.sum((fil_X>1.01*self.lower).double())==half_len
        upper_exist = torch.sum((torch.abs(fil_X-self.upper)<=torch.abs(0.01*self.upper)).double())>0
        upper_cons = torch.sum((fil_X<0.99*self.upper).double()) - num_lowest==half_len
        bound = count_less==half_len
        # 0.02 since 2*0.01
        bound_avg = (torch.abs(self.lower+self.upper-2*self.w)<=torch.abs(0.02*self.w))

        median_in_cons = torch.logical_and(less_cons, more_cons)
        median_out_cons = torch.logical_and(torch.logical_and(bound, bound_avg), torch.logical_and(torch.logical_and(lower_cons, upper_cons), torch.logical_and(lower_exist, upper_exist)))

        return(torch.where(count_equal==0, median_out_cons, median_in_cons), self.w)


prover_gen_settings(data_path,selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

Theory_output:  tensor(40., dtype=torch.float64)
==== Generate & Calibrate Setting ====
scale:  [2]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":2,"param_scale":2,"scale_rebase_multiplier":10,"lookup_range":[-582,1208],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":14432,"total_assignments":15925,"total_const_size":2126,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,2],"model_input_scales":[2],"module_sizes":{"kzg":[],"poseidon":[14432,[1]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"Div":{"denom":2.0}},"ReLU",{"Floor":{"scale":4.0}},{"GreaterThan":{"a":0.0}},"KroneckerDelta"],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [41]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
print("Theory output: ", theory_output)
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

spawning module 0
spawning module 2


==== setting up ezkl ====


spawning module 0
spawning module 2


Time setup: 2.2689290046691895 seconds
Theory output:  tensor(40., dtype=torch.float64)
!@# compiled_model exists? True
!@# compiled_model exists? True
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 40.0
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12341676197686541490, 2627393525778350065, 16625494184434727973, 1478518078215075360]]], 'proof': '1bf314349fbcf7a868f9bca361ec5c4adb7c91373fcd803011948b0dd88a5a8302b6e8eb067975b24f1dee1a21df86132539938e2cf88930a9bdaec21f573da717d359f6147239c4a4980800052e9125382810f73e7b5d0968e4c49789bb2d9316a361ae5eabbcd4573ab22558be5d830bf9138d140714a031b4491a36cf673726ce7ba4d1ef9856d5d7b3f61b48dfce57fbb75ebb012985e19971f50bf6d4b00eacf425d8151670bf72f55a38a5903458597389716c4065eeb4873f7ae18b462c3264ef0f52596d16fd996d5636df8488836898a8881a070df2ec25a5154b3d02289417a6e19879bddc97c782296078cae4b0a49d310f84f7d113c81d99066217352f0e1c53bf38a69624719dde6640c85f525e1f9a6c4508b28b3a9945db0b0dd43f9d21ca23561a124354bb0dc686cfd76adb6afb5550e948a74b4ef622e90bfefe71c50dcaed1f09c7c4b046c39e748fe837873d7ca0d961443f595ea7

In [42]:
# Verifier verifies
verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)

num_inputs:  1
prf instances:  [[[1780239215148830498, 13236513277824664467, 10913529727158264423, 131860697733488968], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [12341676197686541490, 2627393525778350065, 16625494184434727973, 1478518078215075360]]]
proof boolean:  1.0
proof result 1 : 40.0
verified
